In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from warnings import simplefilter,filterwarnings
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
import matplotlib.ticker as mticker
import matplotlib.ticker as ticker
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import scipy.io
import pickle
import sys
import gc
import os
import re
filterwarnings('ignore')
filterwarnings(action='once')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
simplefilter('ignore', FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS'
code_path = os.path.join(root_path, 'Codes','Shared Codes')
data_path =  os.path.join(root_path, 'data')
results_path = os.path.join(root_path, 'Results')
image_path = os.path.join(root_path, 'Images')
csv_path = os.path.join(root_path, 'CSV')

folders_evolving = ["kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest",
                    "kpfaoselm_gp_combiner_with_model_replacement_evolving_DFS",
                    "kpfaoselm_gp_combiner_with_model_replacement_evolving",
                    "kpfaoselm_gp_combiner_with_model_replacement_evolving_IssamDFS",
                    ]

folders_without_evolving = ["kpfaoselm_gp_combiner_with_model_replacement_random_forest",
                            "kpfaoselm_gp_combiner_with_model_replacement_DFS",
                            "kpfaoselm_gp_combiner_with_model_replacement",
                            "kpfaoselm_gp_combiner_with_model_replacement_IssamDFS",
                            ]
algo_name = 'kpfaoselm_gp'


# folders_evolving = ["faoselm_gp_combiner_with_model_replacement_evolving_random_forest",
#                     "faoselm_gp_combiner_with_model_replacement_evolving_DFS",
#                     "faoselm_gp_combiner_with_model_replacement_evolving",
#                     "faoselm_gp_combiner_with_model_replacement_evolving_IssamDFS",
#                     ]
# folders_without_evolving = ["faoselm_gp_combiner_with_model_replacement_random_forest",
#                             "faoselm_gp_combiner_with_model_replacement_DFS",
#                             "faoselm_gp_combiner_with_model_replacement",
#                             "faoselm_gp_combiner_with_model_replacement_IssamDFS",
#                             ]
# algo_name = 'faoselm_gp'


# folders_evolving = ["gp_combiner_with_model_replacement_evolving_random_forest",
#                     "gp_combiner_with_model_replacement_evolving_DFS",
#                     "gp_combiner_with_model_replacement_evolving",
#                     "gp_combiner_with_model_replacement_evolving_IssamDFS",
#                     ]
# folders_without_evolving = ["gp_combiner_with_model_replacement_random_forest",
#                             "gp_combiner_with_model_replacement_DFS",
#                             "gp_combiner_with_model_replacement",
#                             "gp_combiner_with_model_replacement_IssamDFS",
#                             ]
# algo_name = 'gp'


sys.path.insert(0,code_path)
from ensemble import Ensemble, Classifier

In [ ]:
filenames = ['ISCX2012.csv', 'CSE-CIC2018.csv', 'CICIDS2017.csv', 'kddcup99_csv.csv', 'Blip_Drift.csv',
             'Gradual_Drift.csv', 'Incremental_Drift.csv', 'Recurrent_Drift.csv', 'Sudden_Drift.csv']
files = list(map(lambda x: os.path.join(data_path, x), filenames))

In [ ]:
def save_object(obj, filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

In [ ]:
def load_object(filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [ ]:
def save_pickle(obj, file_name):
  with open(file_name, 'wb') as f:
    pickle.dump(obj, f)
def load_pickle(file_name):
  with open(file_name, 'rb') as f:
    d = pickle.load(f)
  return d

In [ ]:
def plot_results(metric_name, methods_name, results, colors, data_name, algo_name, drift_location=None, Name="", step=1):
    plt.figure(figsize=(25, 8))
    plt.title(data_name+" "+Name)
    numerecal_results = pd.DataFrame()
    # for k, result in enumerate(results):
    #     print(k, methods_name[k], data_name, len(result))
    #     if not drift_location is None:
    #       print(drift_location[k])
    for k, result in enumerate(results):
        # x = np.arange(step, len(result) + step, step)
        # y = result[::step]

        x = [i for i, x in enumerate(results[k]) if i % step == 0]
        y = [x for i, x in enumerate(results[k]) if i % step == 0]

        numerecal_results[methods_name[k]] = y
        plt.plot(x, y, color=colors[k], label=methods_name[k])
        plt.scatter(x, y, color=colors[k], s=20)  # Add dots to the plot
        if not drift_location is None:
          for loc in drift_location[k]:
            if type(loc) is str : continue
            loc = loc-1
            y_value = y[loc]
            plt.plot(loc, y_value, marker="o", markersize=9, markerfacecolor=colors[k])
    plt.legend(loc="best")
    plt.xlabel('Chunk number')
    plt.ylabel(metric_name)

    ax = plt.gca()
    ax.locator_params(axis='y', nbins=15)
    ax.yaxis.set_major_locator(MaxNLocator(nbins=15))

    num_ticks = 50  # Change this value to adjust the number of ticks
    ax.xaxis.set_major_locator(plt.MaxNLocator(num_ticks))

    ax.xaxis.set_major_formatter(plt.ScalarFormatter(useOffset=False, useMathText=True))
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
    plt.xticks(rotation=45, fontsize='xx-small')
    plt.grid()

    # Format x-axis ticks as integers
    plt.gca().xaxis.set_major_locator(plt.MultipleLocator(step))
    plt.gca().xaxis.set_major_formatter(plt.ScalarFormatter(useOffset=False, useMathText=True))
    my_path = os.path.join(image_path,"{}".format(data_name))
    os.makedirs(my_path, exist_ok=True)
    numerecal_results.to_csv(os.path.join(my_path,"{}.csv".format(algo_name+"_"+metric_name+"_"+Name)), index=False)
    plt.savefig(os.path.join(my_path, algo_name+"_"+metric_name+"_"+Name))
    plt.savefig(os.path.join(my_path, algo_name+"_"+metric_name+"_"+Name+'.svg'), format='svg')
    plt.show()

In [ ]:
def plot_dimensional_reduction(metric_name, methods_name, results, colors, data_name,drift_location=None, Name="", step=1):
  barWidth = 0.25
  fig = plt.subplots(figsize=(15, 5))
  numerecal_results = pd.DataFrame()
  plt.title(data_name+" "+Name)
  br1 = np.arange(len(results[0]))
  br = [br1]
  for i in results[:-1]:
    br1 = [x + barWidth for x in br1]
    br.append(br1)

  for i, res in enumerate(results):
    numerecal_results[methods_name[i]] = res
    plt.bar(br[i], res, color=colors[i], width=barWidth, edgecolor=colors[i], label=methods_name[i])

  plt.legend(loc="best")
  plt.xlabel('Chunk number')
  plt.ylabel(metric_name)

  plt.xticks(rotation=45, fontsize='xx-small')
  plt.gca().xaxis.set_major_locator(plt.MultipleLocator(step))
  plt.gca().xaxis.set_major_formatter(plt.ScalarFormatter(useOffset=False, useMathText=True))
  my_path = os.path.join(image_path,"{}".format(data_name))
  os.makedirs(my_path, exist_ok=True)
  numerecal_results.to_csv(os.path.join(my_path,"{}.csv".format(metric_name+"_"+Name)), index=False)
  plt.savefig(os.path.join(my_path, metric_name+"_"+Name))
  plt.savefig(os.path.join(my_path, metric_name+"_"+Name+'.svg'), format='svg')
  plt.show()

In [ ]:
def Finall_Results(folders,Name=""):
  df_accuracy, df_precision , df_recall , df_f1_score, df_auc, df_drift_location = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
  method_names = ["GPC-KOS-RF","GPC-KOS-DFS", "GPC-OS","GPC-KOS-DQN-MAFS"]
  # method_names = ["GPC-FOS-RF","GPC-FOS-DFS", "GPC-OS","GPC-FOS-DQN-MAFS"]
  # method_names = ["GPC-OS-RF","GPC-OS-DFS", "GPC-OS","GPC-OS-DQN-MAFS"]


  synthetic_data = ['Blip', 'GradualDrift', 'Incremental', 'Recurrent', 'SuddenDrift']

  data_names = ['ISCX2012', 'CSE-CIC2018', 'CICIDS2017', 'KddCup99', 'Blip',
                'GradualDrift', 'Incremental', 'Recurrent', 'SuddenDrift']


  df_accuracy['data_names'],df_precision['data_names'] =  data_names, data_names
  df_f1_score['data_names'],df_recall['data_names'] = data_names, data_names
  df_auc['data_names'], df_drift_location['data_names'] = data_names, data_names


  metrices = ["accuracy", "precision", "recall", "f1-score", "auc"]
  # colors = ['g', 'r', 'cyan', 'gray', 'y','blue','black','orange','magenta']
  colors = ['g', 'r', 'b','orange']

  for i, data in enumerate(filenames):
    accuracys, precisions, recalls, f1_scores, aucs, drift_location, memory_reduction, prediction_times = [], [], [], [], [], [], [], []
    for j,folder in enumerate(folders):
      results = load_object(data+"_ensemble",os.path.join(os.path.join(results_path,folder),data)).scores
      accuracy, precision , recall , f1_score, auc = [], [], [], [], []
      for key in results.keys():
        result = results[key]
        accuracy.append(result[metrices[0]])
        precision.append(result[metrices[1]])
        recall.append(result[metrices[2]])
        f1_score.append(result[metrices[3]])
        auc.append(result[metrices[4]])

      accuracys.append(accuracy)
      precisions.append(precision)
      recalls.append(recall)
      f1_scores.append(f1_score)
      aucs.append(auc)
      if data_names[i] in synthetic_data: drift_location.append([25,50,75])
      else:
        drift_location.append(list(load_object(data+"_drift_location",os.path.join(os.path.join(results_path,folder),data)).keys()))

      memory_reduction.append(list(load_object(data+"_memory_reduction",os.path.join(os.path.join(results_path,folder),data)).values()))
      prediction_times.append(list(load_object(data+"_prediction_times",os.path.join(os.path.join(results_path,folder),data)).values()))


    # plot_dimensional_reduction(metrices[0], method_names, accuracys, colors, data_names[i], drift_location, Name, step=1)
    # plot_dimensional_reduction(metrices[3], method_names, f1_scores, colors, data_names[i], drift_location, Name, step=1)
    # plot_dimensional_reduction(metrices[1], method_names, precisions, colors, data_names[i], drift_location, Name, step=1)
    # plot_dimensional_reduction(metrices[2], method_names, recalls, colors, data_names[i], drift_location, Name, step=1)
    # plot_dimensional_reduction(metrices[4], method_names, aucs, colors, data_names[i], drift_location, Name, step=1)
    # plot_dimensional_reduction("Memory Reduction", method_names, memory_reduction, colors, data_names[i], Name=Name, step=1)
    # plot_dimensional_reduction("Prediction Times", method_names, prediction_times, colors, data_names[i], Name=Name, step=1)


    for accuracy, precision , recall , f1_score, auc ,method_name in  zip(accuracys, precisions, recalls, f1_scores, aucs, method_names):
      df_accuracy.at[i,method_name] =  np.average(accuracy)
      df_precision.at[i,method_name] =  np.average(precision)
      df_recall.at[i,method_name] = np.average(recall)
      df_f1_score.at[i,method_name] =  np.average(f1_score)
      df_auc.at[i,method_name] = np.average(auc)
    plot_results(metrices[0], method_names, accuracys, colors, data_names[i], algo_name, drift_location, Name, step=1)
    plot_results(metrices[3], method_names, f1_scores, colors, data_names[i], algo_name, drift_location, Name, step=1)
    plot_results(metrices[1], method_names, precisions, colors, data_names[i], algo_name, drift_location, Name, step=1)
    plot_results(metrices[2], method_names, recalls, colors, data_names[i], algo_name, drift_location, Name, step=1)
    plot_results(metrices[4], method_names, aucs, colors, data_names[i], algo_name, drift_location, Name, step=1)
    plot_results("Memory Usage", method_names, memory_reduction, colors, data_names[i], algo_name, Name=Name, step=1)
    plot_results("Prediction Times", method_names, prediction_times, colors, data_names[i], algo_name, Name=Name, step=1)

  df_auc.to_csv(os.path.join(csv_path,algo_name+"_auc_"+Name.lower()+".csv"), index=False)
  df_f1_score.to_csv(os.path.join(csv_path,algo_name+"_f1_score_"+Name.lower()+".csv"), index=False)
  df_recall.to_csv(os.path.join(csv_path,algo_name+"_recall_"+Name.lower()+".csv"), index=False)
  df_precision.to_csv(os.path.join(csv_path,algo_name+"_precision_"+Name.lower()+".csv"), index=False)
  df_accuracy.to_csv(os.path.join(csv_path,algo_name+"_accuracy_"+Name.lower()+".csv"), index=False)

In [ ]:
Finall_Results(folders=folders_evolving,Name="With Evolving")

In [ ]:
Finall_Results(folders=folders_without_evolving,Name="Without Evolving")